# Generates the input histograms to use with TRexFitter
## 1. Gather files from paths.
## 2. Correct Z-QCDjj uncertainty taking into account the uncertainties coming from the RW.
## 3. OPTIONAL -> Add systematics.
## 4. Create file and upload it to lxplus.

## 1

In [14]:
import ROOT as r
import os
import numpy as np
import matplotlib.pyplot as plt

def model_uncern(covariance_matrix,x):
    var_a = covariance_matrix[0][0]
    var_b = covariance_matrix[1][1]
    var_c = covariance_matrix[2][2]
    
    cov_ab = covariance_matrix[0][1]
    cov_ac = covariance_matrix[0][2]
    cov_bc = covariance_matrix[1][2]
    
    diagonal_term = var_a*(x**4) + var_b*(x**2) + var_c
    no_diagonal_term = 2*cov_ab*(x**3) + 2*cov_ac*(x**2) + 2*cov_bc*x
    
    return np.sqrt(diagonal_term+no_diagonal_term)

def parabolic_shape(mjj,parameters):
    a = parameters[0]
    b = parameters[1]
    c = parameters[2]
    return a*mjj*mjj+b*mjj+c

model_uncer = np.vectorize(model_uncern)
parabolic_shape = np.vectorize(parabolic_shape,excluded=['parameters'])

fitParams = {"Sherpa": [ 1.31867345e-07, -6.91194368e-04,  1.49568233e+00], 
              "MG" : [ 1.35194973e-07, -5.44814667e-04,  9.89623841e-01] }

covarianceMatrix =  {"Sherpa" : [[ 1.19177067e-17, -2.94876918e-14,  1.41446066e-11],
                               [-2.94876918e-14,  8.14817316e-11, -4.30728335e-08],
                               [ 1.41446066e-11, -4.30728335e-08,  2.66545658e-05]],
                    "MG" : [[ 6.10103774e-18, -1.39152060e-14,  6.28353598e-12],
                           [-1.39152060e-14,  3.55524563e-11, -1.77055103e-08],
                           [ 6.28353598e-12, -1.77055103e-08,  1.02381778e-05]]}

def scaleBinUncertainty(histogram,sampleName):
    
    sampleType = "MG"
    if "Sherpa" in sampleName:
        sampleType = "Sherpa"
    
    for i in range(1,histogram.GetNbinsX()+1):
        x = histogram.GetBinCenter(i)
        error = histogram.GetBinError(i)
        rw = parabolic_shape(x,parameters=fitParams[sampleType])
        rw_error = model_uncern(covarianceMatrix[sampleType],x)
        newError = np.sqrt(error**2 + ((rw_error/rw)**2)*(error**2))
        histogram.SetBinError(i,newError)

def channel(path):
    if "MuMu" in path:
        return "Zmumu"
    elif "Zee" in path:
        return "Zee"
    elif "TauTau" in path or "MuEle" in path:
        return "Ztautau"
    elif "Zll" in path:
        return "Zll"
    else :
        raise ValueError("Path not found")

remotePath = "/afs/cern.ch/work/d/dbaronmo/private/Fitter/"
localPath = "/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau/AnalysisCrossCheck/SR/"

dataSamples = ['Data.root']
signalSamples = ['Signal_Sherpa.root','Signal_PoPy.root']
qcdSamples = [channel(localPath)+i for i in ["_SherpaRW.root","_MGRW.root","_Average_EW-Sherpa.root","_Average_EW-PoPy.root"]]
backgroundSamples = ['Wjets.root','VV.root',"ttbar.root",'singletop.root']
if "Tau" in localPath or "MuEle" in localPath:
    backgroundSamples += ['Higgs.root','Zjets.root','MJ.root']
    
samples = dataSamples+signalSamples+qcdSamples+backgroundSamples

## 2

In [15]:
histogramNames = ["mass_jj"]

outputFile = r.TFile.Open(channel(localPath)+"histograms.root","RECREATE")

for sample in samples:
    for histogramName in histogramNames:
        file = r.TFile.Open(localPath+sample,"READ")
        histogram = file.Get(histogramName)
        
        if "RW" in sample:
            scaleBinUncertainty(histogram,sample)
        
        # Include NOMINAL tag to samples used for systematics    
        if "Signal_Sherpa" in sample[:-5] or "SherpaRW" in sample[:-5]:
            outputFile.WriteObject(histogram,histogramName+"_"+sample[:-5]+'_NOMINAL')
        else :
            outputFile.WriteObject(histogram,histogramName+"_"+sample[:-5])
        file.Close()
        
outputFile.Close()

# 3 (OPTIONAL) --- SYSTEMATICS

In [16]:
from SystematicsAdd import getSystematicsName

systematicsPath = '/Users/diegomac/Documents/HEP/VBF-Analysis/TauTau//AnalysisCrossCheck/Systematics/'
systematicSamples = ['Signal_Sherpa.root',channel(systematicsPath)+'_SherpaRW.root']

# Extract the systematics names
systematicNames = getSystematicsName(systematicsPath,'mass_jj')
systematicShapeNames = systematicNames['shapeSys']
systematicWeightNames = systematicNames['weightSys']

# Clean tags (for Z->ll) samples
tags = ['elec_X','elec_Y','muon_X','muon_Y']
removeNames = []
for systematicName in systematicWeightNames:
    for tag in tags:
        if tag in systematicName:
            removeNames.append(systematicName)
for removeSys in removeNames:
    print('Removing ... ',removeSys)
    systematicWeightNames.remove(removeSys)

# Open the output file        
outputFile = r.TFile.Open(channel(localPath)+"histograms.root","UPDATE")

# Clean the nominal samples
#for sample in systematicSamples:
    #print("mass_jj"+sample[:-5]+";*")
    #outputFile.Delete("mass_jj_"+sample[:-5]+";*")

# Fill shape systematics (NOMINAL Shape is extracted here)
for sample in systematicSamples:
    
    for systematicName in systematicShapeNames:
        file = r.TFile.Open(systematicsPath+sample[:-5]+'_'+systematicName+'.root',"READ")
        histogram = file.Get("mass_jj")

        if "RW" in sample:
            scaleBinUncertainty(histogram,sample)
            
        if "NOMINAL" in systematicName:
            outputFile.WriteObject(histogram,"mass_jj"+"_"+sample[:-5])
        else:
            outputFile.WriteObject(histogram,"mass_jj"+"_"+sample[:-5]+'_'+systematicName)

# Fill weight systematics        
for sample in systematicSamples:
    
    for systematicName in systematicWeightNames:
        file = r.TFile.Open(systematicsPath+sample[:-5]+'_NOMINAL.root',"READ")
        histogram = file.Get("mass_jj"+'_'+systematicName)

        if "RW" in sample:
            scaleBinUncertainty(histogram,sample)

        outputFile.WriteObject(histogram,"mass_jj"+"_"+sample[:-5]+'_'+systematicName)
        
        
outputFile.Close()

# 4

In [17]:
exitStatus = os.system("scp "+channel(localPath)+"histograms.root "+"dbaronmo@lxplus.cern.ch:"+remotePath+".")
if exitStatus==0:
    print("Sample Uploaded!")

Sample Uploaded!


In [23]:
# Print the folllowing config for TRex-Fitter
'''
Systematic: "MUON_MS"
  Title: "MUON_MS"
  Type: HISTO
  HistoNameSufUp: "_MUON_MS_1up"
  HistoNameSufDown: "_MUON_MS_1down"
  Symmetrisation: TWOSIDED
  Samples: "Signal_Sherpa","Zmumu_SherpaRW"
'''

samplesString = ''
for sample in systematicSamples:
    samplesString+='"'+sample[:-5]+'",'
samplesString = samplesString[:-1]

print("Printing ",len(systematicShapeNames+systematicWeightNames)," systematics.\n")

for systematicName in systematicWeightNames:
    if "_1up" in systematicName:
        print("Systematic: "+'"'+systematicName.replace('_1up','')+'"')
        print(' Title: '+'"'+systematicName.replace('_1up','')+'"')
        print(' Type : HISTO')
        print(' HistoNameSufUp: '+'"_'+systematicName+'"')
        print(' HistoNameSufDown: '+'"_'+systematicName.replace('_1up','_1down')+'"')
        print(' Symmetrisation: TWOSIDED')
        print(' Samples: '+samplesString)
        print('\n')
    
for systematicName in systematicShapeNames:
    if "_1up" in systematicName:
        print("Systematic: "+'"'+systematicName[:-len("_1up")]+'"')
        print(' Title: '+'"'+systematicName[:-len("_1up")]+'"')
        print(' Type : HISTO')
        print(' HistoNameSufUp: '+'"_'+systematicName[:-len("_1up")]+'_1up"')
        print(' HistoNameSufDown: '+'"_'+systematicName[:-len("_1up")]+'_1down"')
        print(' Symmetrisation: TWOSIDED')
        print(' Samples: '+samplesString)
        print('\n')

Printing  158  systematics.

Systematic: "tau_0_TAUS_TRUEHADTAU_EFF_RNNID_1PRONGSTATSYSTPT3040_TauEffSF_JetRNNmedium"
 Title: "tau_0_TAUS_TRUEHADTAU_EFF_RNNID_1PRONGSTATSYSTPT3040_TauEffSF_JetRNNmedium"
 Type : HISTO
 HistoNameSufUp: "_tau_0_TAUS_TRUEHADTAU_EFF_RNNID_1PRONGSTATSYSTPT3040_1up_TauEffSF_JetRNNmedium"
 HistoNameSufDown: "_tau_0_TAUS_TRUEHADTAU_EFF_RNNID_1PRONGSTATSYSTPT3040_1down_TauEffSF_JetRNNmedium"
 Symmetrisation: TWOSIDED
 Samples: "Signal_Sherpa","Ztautau_SherpaRW"


Systematic: "jet_JET_JvtEfficiency_central_jets_global_ineffSF_JVT"
 Title: "jet_JET_JvtEfficiency_central_jets_global_ineffSF_JVT"
 Type : HISTO
 HistoNameSufUp: "_jet_JET_JvtEfficiency_1up_central_jets_global_ineffSF_JVT"
 HistoNameSufDown: "_jet_JET_JvtEfficiency_1down_central_jets_global_ineffSF_JVT"
 Symmetrisation: TWOSIDED
 Samples: "Signal_Sherpa","Ztautau_SherpaRW"


Systematic: "tau_0_TAUS_TRUEHADTAU_EFF_RNNID_SYST_TauEffSF_JetRNNmedium"
 Title: "tau_0_TAUS_TRUEHADTAU_EFF_RNNID_SYST_TauEffSF_J